# Delta Factor Analysis for Bulk DataSet

In [1]:
%load_ext autoreload
from pathlib import Path
# Get submodule directory
RESULTS_ROOT = Path.cwd().parent.absolute() / 'tb_results'
RESULTS_ROOT

PosixPath('/Users/alexanderbuccheri/Codes/tb_benchmarking/tb_results')

In [3]:
## %autoreload
"""Delta Factor Functions

Load all functions once.
"""
# %autoreload
import json
import numpy as np
import pandas as pd
from typing import Union

from tb_lite.crystal_references.crystal_systems import bulk_materials
from delta_factor.loader import tblite_loader, qe_loader
from delta_factor.fit_eos import delta_factor, EVCurveData, calculate_birch_murnaghan_relation


# Map E vs V file name to the internal material labelling we use....
# Because the file names don't all match the materials labels we use internally
output_to_internal_label = {'bn_cubic': 'bn_cubic', 'bn_hex': 'bn_hex', 'cdse': 'cdse', 'diamond': 'diamond', 'gaas': 'gaas', 'gan': 'gan',
                            'ge': 'germanium', 'graphite': 'graphite', 'mgo': 'mgo', 'mos2': 'mos2', 'nacl': 'nacl', 'pbs': 'pbs',
                            'si': 'silicon', 'tio2_ana': 'tio2_ana', 'tio2_rutile': 'tio2_rutile', 'ws2': 'ws2',
                            'zno': 'zinc_oxide', 'zro2': 'zro2', 'wo3_monoclinic': 'wo3_monoclinic'}


def relative_error(estimated, mean):
    """ Relative error as a percentage
    """
    return (np.abs(estimated - mean) / mean) * 100


def fit_model(result: Union[np.ndarray, EVCurveData]) -> EVCurveData:
    """ Light wrapper for fitting birch_murnaghan model to the data
    """
    result_obj = result

    if not isinstance(result, EVCurveData):
        result_obj= EVCurveData(result[:, 0], result[:, 1])

    return calculate_birch_murnaghan_relation(result_obj)


def compute_delta_factor(materials: list, tb_results: dict, qe_results: dict) -> dict:
    """ Compute delta factor between tight-binding data and QE data.
    """
    ev_to_mev = 1000.
    vol_tb = []
    vol_qe = []
    vol_error = []
    delta_f = []

    get_min_volume = lambda result: np.amin(result.volume) if isinstance(result, EVCurveData) else np.amin(result[:, 0])

    for material in materials:
        fit_succeeded = True

        try:
            fit1_data: EVCurveData = fit_model(tb_results[material])
        except RuntimeError:
            print(f"TB fitting attempts exceeded for {tb_results['label']}  {material}")
            fit_succeeded = False
        try:
            fit2_data: EVCurveData = fit_model(qe_results[material])
        except RuntimeError:
            print(f"QE fitting attempts exceeded for {qe_results['label']} {material}")
            fit_succeeded = False

        if fit_succeeded:
            tb_equilibrium_vol = fit1_data.fitted_parameters['eq_volume']['value']
            dft_equilibrium_vol = fit2_data.fitted_parameters['eq_volume']['value']
        else:
            # Fall back to discrete data
            tb_equilibrium_vol = get_min_volume(tb_results[material])
            dft_equilibrium_vol = get_min_volume(qe_results[material])

        # Collate results
        rel_diff_vol = relative_error(tb_equilibrium_vol, dft_equilibrium_vol)
        vol_tb.append(tb_equilibrium_vol)
        vol_qe.append(dft_equilibrium_vol)
        vol_error.append(relative_error(tb_equilibrium_vol, dft_equilibrium_vol))

        atoms = bulk_materials[output_to_internal_label[material]]
        n_atoms = len(atoms.get_atomic_numbers())

        if fit_succeeded:
            delta_f.append(delta_factor(fit1_data, fit2_data, dft_equilibrium_vol) / float(n_atoms)  * ev_to_mev)
        else:
            delta_f.append(np.infty)

    # Package for dataframe
    data = {rf"{tb_results['label']} Equilibrium volume $\mathring{{A}}\,^3$": np.around(vol_tb, decimals=2),
            rf"{qe_results['label']} Equilibrium volume $\mathring{{A}}\,^3$": np.around(vol_qe, decimals=2),
            f"Error in Volume w.r.t. {qe_results['label']} (%)": np.around(vol_error, decimals=2),
            "Delta Factor (meV /atom)": np.around(delta_f, decimals=2)}

    return data


def compute_bulk_modulus(materials: list, tb_results: dict, qe_results: dict) -> dict:
    """ Compute bulk moduli for tight-binding data and QE data.
    
    Note, one could return it in the function above, but as this costs nothing to 
    compute, I've written a separate function.
    
    Units:
    Expect energy in eV and volume in Angstrom^3
    Bulk modulus is E/V, so to convert to GaP:
     eV_to_J / (angstrom_to_m^3 * 10^9) == eV_to_J * 10^21
    """
    fit1_bm = []
    fit2_bm = []
    fit1_error = []
    
    to_GPa = 1.602177e-19 * 1.e21
    
    for material in materials:
        fit_succeeded = True

        try:
            fit1_data: EVCurveData = fit_model(tb_results[material])
        except RuntimeError:
            print(f"TB fitting attempts exceeded for {tb_results['label']}  {material}")
            fit_succeeded = False
        try:
            fit2_data: EVCurveData = fit_model(qe_results[material])
        except RuntimeError:
            print(f"QE fitting attempts exceeded for {qe_results['label']} {material}")
            fit_succeeded = False

        if fit_succeeded:
            fit1_bm.append(fit1_data.fitted_parameters['bulk_modulus']['value'] * to_GPa)
            fit2_bm.append(fit2_data.fitted_parameters['bulk_modulus']['value'] * to_GPa)
            fit1_error.append(relative_error(fit1_bm[-1], fit2_bm[-1]))
        else:
            fit1_bm.append(np.infty)
            fit2_bm.append(np.infty)
            fit1_error.append(np.infty)

    # Package for dataframe
    data = {rf"{tb_results['label']} Bulk Modulus (GaP)": np.around(fit1_bm, decimals=2),
            rf"{qe_results['label']} Bulk Modulus (GaP)": np.around(fit2_bm, decimals=2),
            f"{tb_results['label']} Error in Bulk Modulus w.r.t. {qe_results['label']} (%)": np.around(fit1_error, decimals=2)
            }

    return data



In [4]:
""" Delta factor: TB lite 1 vs QE
"""
materials = ['bn_cubic', 'bn_hex', 'cdse', 'diamond', 'gaas', 'gan', 'ge', 'graphite', 'mgo', 'mos2',
             'nacl', 'pbs', 'si', 'tio2_rutile', 'ws2', 'zno', 'zro2']
no_fit = ['tio2_ana', 'wo3_monoclinic']

tb_results = tblite_loader(RESULTS_ROOT, materials)
qe_results = qe_loader(RESULTS_ROOT, materials)
tb_results['label'] = 'TB Lite 1'
qe_results['label'] = 'QE'

data1 = compute_delta_factor(materials, tb_results, qe_results)
df1 = pd.DataFrame(data=data1)
df1 = df1.rename(index={i: name for i, name in enumerate(materials)})

print(df1)

             TB Lite 1 Equilibrium volume $\mathring{A}\,^3$  \
bn_cubic                                               11.47   
bn_hex                                                 42.17   
cdse                                                  118.73   
diamond                                                11.11   
gaas                                                   47.55   
gan                                                    28.63   
ge                                                     45.76   
graphite                                               45.12   
mgo                                                    19.47   
mos2                                                  136.67   
nacl                                                   92.00   
pbs                                                    49.31   
si                                                     43.37   
tio2_rutile                                            57.08   
ws2                                     

In [5]:
# Delta factor stats
deltas = df1.loc[:, 'Delta Factor (meV /atom)'].tolist()
print('min, mean, median, max')
print(np.amin(deltas), np.around(np.mean(deltas), 2), np.median(deltas), np.amax(deltas))

min, mean, median, max
1.84 119.1 29.32 966.08


In [6]:
"""Bulk Moduli
"""
from nb_modules.qcore_loaders import qcore_repack


materials = ['bn_cubic', 'bn_hex', 'cdse', 'diamond', 'gaas', 'gan', 'ge', 'graphite', 'mgo', 'mos2',
             'nacl', 'pbs', 'si', 'tio2_rutile', 'wo3_monoclinic', 'ws2', 'zno', 'zro2']
no_fit = ['tio2_ana']


# TB Lite
tb_results = tblite_loader(RESULTS_ROOT, materials)
tb_results['label'] = 'TB Lite 1'

# QE
qe_results = qe_loader(RESULTS_ROOT, materials)
qe_results['label'] = 'QE'

# QCore
with open(Path(RESULTS_ROOT, "qcore/bulk_system_at_renormalized_cutoff.json"), 'r') as fid:
    qcore_raw_data = json.load(fid)

qcore_results = qcore_repack(qcore_raw_data['40'], output='np')
qcore_results_20 = qcore_repack(qcore_raw_data['20'], output='np')

qcore_results['tio2_rutile'] = qcore_results_20.pop('tio2_rutile')
qcore_results['nacl'] = qcore_results_20.pop('nacl')
qcore_results['label'] = 'QCore'
qcore_results_20.clear()

# Compute bulk moduli
data_bm_lite_qe = compute_bulk_modulus(materials, tb_results, qe_results)
data_bm_qcore_qe = compute_bulk_modulus(materials, qcore_results, qe_results)
assert len(data_bm_lite_qe['TB Lite 1 Bulk Modulus (GaP)']) == len(data_bm_qcore_qe['QCore Bulk Modulus (GaP)'])
data_bm = data_bm_lite_qe | data_bm_qcore_qe

# Output as pandas DF
df_bm = pd.DataFrame(data=data_bm)
df_bm = df_bm.rename(index={i: name for i, name in enumerate(materials)})

# Mean relative errors
tb_bulk_moduli = data_bm['TB Lite 1 Error in Bulk Modulus w.r.t. QE (%)']
print('Mean relative errors')
print('TBLite:', np.mean(tb_bulk_moduli[tb_bulk_moduli != np.inf]))
print('QCore:', np.mean(data_bm['QCore Error in Bulk Modulus w.r.t. QE (%)']))
print('Median relative errors')
print('TBLite:', np.median(tb_bulk_moduli[tb_bulk_moduli != np.inf]))
print('QCore:', np.median(data_bm['QCore Error in Bulk Modulus w.r.t. QE (%)']))


TB fitting attempts exceeded for TB Lite 1  wo3_monoclinic
Mean relative errors
TBLite: 51.332941176470584
QCore: 40.07166666666668
Median relative errors
TBLite: 18.81
QCore: 31.265


In [14]:
""" Delta factor: QCore vs QE
"""
from nb_modules.qcore_loaders import load_json_qcore

#Common materials
materials = ['bn_cubic', 'bn_hex', 'cdse', 'diamond', 'gaas', 'gan', 'ge', 'graphite', 'mgo', 'mos2',
             'nacl', 'pbs', 'si', 'tio2_rutile', 'wo3_monoclinic', 'ws2', 'zno', 'zro2']

missing_from_qe = ['cu', 'pbte', 'tio2_ana']

# Comment on Results Files
# Was previously using "qcore/bulk_system_at_renormalized_cutoff.json"
# but asked Rui to do a wider range of volumes w.r.t. the plots: "bulk_system_wider_range.json" (only includes a cutoff of 80)
# After which I asked him to repeat the wider range but with other cutoffs included: "bulk_system_20_and_40_cutoff.json"
# I should combine bulk_system_wider_range.json with bulk_system_20_and_40_cutoff.json
# Note, no difference in numbers for cutoffs >= 40 Bohr

default_optimal_cutoff = {'bn_cubic': '40',
                  'bn_hex': '40',
                  'cdse': '40',
                  'diamond': '40',
                  'gaas': '40',
                  'gan': '40',
                  'germanium': '40',
                  'graphite': '40',
                  'mgo': '40',
                  'mos2': '40',
                  'nacl': '20',
                  'pbs': '40',
                  'silicon': '40',
                  'tio2_rutile': '20',
                  'ws2': '40',
                  'wo3_monoclinic': '40',
                  'zinc_oxide': '40',
                  'zro2': '40'}

# QCore results
qcore_results = load_json_qcore(RESULTS_ROOT, "qcore/bulk_system_20_and_40_cutoff.json", default_optimal_cutoff, output='np')
qcore_results['label'] = 'QCore'

# QE Results
qe_results = qe_loader(RESULTS_ROOT, materials)
qe_results['label'] = 'QE'

data2 = compute_delta_factor(materials, qcore_results, qe_results)
df2 = pd.DataFrame(data=data2)
df2 = df2.rename(index={i: name for i, name in enumerate(materials)})
df2.sort_values('Delta Factor (meV /atom)')

print(df2)

                QCore Equilibrium volume $\mathring{A}\,^3$  \
bn_cubic                                              11.79   
bn_hex                                                43.25   
cdse                                                 118.71   
diamond                                               11.14   
gaas                                                  48.10   
gan                                                   25.83   
ge                                                    48.87   
graphite                                              45.18   
mgo                                                   22.51   
mos2                                                 133.01   
nacl                                                  37.42   
pbs                                                   46.14   
si                                                    44.08   
tio2_rutile                                           61.95   
wo3_monoclinic                                       21

In [13]:
# Delta factor stats
deltas2 = df2.loc[:, 'Delta Factor (meV /atom)'].tolist()
print('min, mean, median, max')
print(np.amin(deltas2), np.mean(deltas2), np.median(deltas2), np.amax(deltas2))

min, mean, median, max
0.32 47.47833333333334 24.354999999999997 230.01


In [15]:
""" Delta factor: QCore vs TB Lite 1
"""
import numpy as np
import pandas as pd

#Common materials
materials = ['bn_cubic', 'bn_hex', 'cdse', 'diamond', 'gaas', 'gan', 'ge', 'graphite', 'mgo', 'mos2',
             'nacl', 'pbs', 'si', 'tio2_rutile', 'ws2', 'zno', 'zro2']


# TB Lite
tb_results = tblite_loader(RESULTS_ROOT, materials)
tb_results['label'] = 'TB Lite 1'

# QCore
with open(Path(RESULTS_ROOT, "qcore/bulk_system_at_renormalized_cutoff.json"), 'r') as fid:
    qcore_raw_data = json.load(fid)
qcore_results = qcore_repack(qcore_raw_data['40'], output='np')
qcore_results['label'] = 'QCore'

data3 = compute_delta_factor(materials, qcore_results, tb_results)
df3 = pd.DataFrame(data=data3)
df3 = df3.rename(index={i: name for i, name in enumerate(materials)})

print(df3)

             QCore Equilibrium volume $\mathring{A}\,^3$  \
bn_cubic                                           11.79   
bn_hex                                             43.26   
cdse                                              118.74   
diamond                                            11.14   
gaas                                               48.13   
gan                                                25.85   
ge                                                 48.86   
graphite                                           45.15   
mgo                                                22.51   
mos2                                              133.08   
nacl                                               36.16   
pbs                                                46.89   
si                                                 43.95   
tio2_rutile                                        59.36   
ws2                                               122.94   
zno                                     

In [ ]:
"""Delta Factor Comparison of the Three Instances

We don't show a code-comparison delta factor table, as in
`Science 351 (6280), aad3000 (2016)` as the average delta factor
across this set of materials is "probably" not very meaningful.
"""

# If one wanted to compare all in one go

# df_delta_compare = pd.DataFrame(data={"TBLite1 vs QE":    data1["Delta Factor (meV /atom)"],
#                                       "QCore vs QE":      data2["Delta Factor (meV /atom)"],
#                                       "QCore vs TBLite1": data3["Delta Factor (meV /atom)"]}
#                                 )
# df_delta_compare = df_delta_compare.rename(index={i: name for i, name in enumerate(materials)})
# df_delta_compare